In [2]:
import requests
from PIL import Image, ImageDraw, ImageFont
import base64
import io

/Users/gokulg.k/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class ImageProcessor:
    """
    Handles image-related operations like preprocessing and manipulation
    """
    @staticmethod
    def preprocess_image(image_path, max_size=200):
        """
        Preprocess an image for API input
        
        Args:
            image_path (str): Path to the image file
            max_size (int): Maximum dimension for resizing
        
        Returns:
            str: Base64 encoded image string
        """
        image = Image.open(image_path)
        
        # Resize image
        ratio = min(max_size / image.size[0], max_size / image.size[1])
        new_size = tuple([int(x * ratio) for x in image.size])
        image = image.resize(new_size, Image.Resampling.LANCZOS)
        image = image.convert('RGB')
        
        # Convert to Base64
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG", quality=50)
        base64_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
        
        return f"data:image/jpeg;base64,{base64_image}"

    @staticmethod
    def overlay_text_on_image(image_content, overlay_text, font_size=24):
        """
        Overlay text on the generated image
        
        Args:
            image_content (bytes): Image content
            overlay_text (str): Text to overlay
            font_size (int): Size of the font
        
        Returns:
            bytes: Image with overlaid text
        """
        # Convert bytes to PIL Image
        image = Image.open(io.BytesIO(image_content))
        # draw = ImageDraw.Draw(image)
        
        # # Use a default font
        # font = ImageFont.load_default()
        
        # # Calculate text position (bottom of the image)
        # text_position = (20, image.height - 100)
        # text_color = (255, 255, 255)  # White text
        
        # # Add a semi-transparent background for readability
        # draw.rectangle([0, image.height - 150, image.width, image.height], fill=(0, 0, 0, 128))
        
        # # Draw text
        # draw.text(text_position, overlay_text, font=font, fill=text_color)
        
        # Convert back to bytes
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        return img_byte_arr.getvalue()

In [4]:
from io import BytesIO
class BrandAnalyzer:
    """
    Handles brand asset analysis using LLava-Next API
    """
    def __init__(self, llava_token):
        #Replace the URL with llava model
        self.LLAVA_URL = "https://supplement-convenient-scores-forgotten.trycloudflare.com/llava/generate"
        self.LLAVA_TOKEN = llava_token

    def analyze_brand_assets(self, brand_logo_path, brand_description):
        """
        Analyze brand assets using LLava-Next to extract key visual and textual insights
        
        Args:
            brand_logo_path (str): Path to brand logo image
            brand_description (str): Initial brand description
        
        Returns:
            dict: Insights about brand identity
        """
        # Preprocess brand logo
        #base64_logo = ImageProcessor.preprocess_image(brand_logo_path)

        image = Image.open(brand_logo_path)
        
        # Resize and compress the image
        max_size = 200  # Reduce maximum size for smaller Base64 string
        ratio = min(max_size / image.size[0], max_size / image.size[1])
        new_size = tuple([int(x * ratio) for x in image.size])
        image = image.resize(new_size, Image.Resampling.LANCZOS)
        image = image.convert('RGB')
        
        # Convert image to Base64
        buffer = BytesIO()
        image.save(buffer, format="JPEG", quality=50)  # Reduce quality for smaller size
        base64_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
        
        # Prepare the inputs
        image_string = f"data:image/jpeg;base64,{base64_image}"
        
        # Prepare LLava-Next request
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.LLAVA_TOKEN}"
        }
        prompt = f"<|im_start|>system\nAnalyze these and extract information.<|im_end|><|im_start|>user\n![]({image_string})\n{brand_description}<|im_end|><|im_start|>assistant"

        # prompt = f"""<|im_start|>system
        # Analyze the logo and description. Extract visual elements from Brand Logo and brand personality traits that can be used for marketing content.
        # <|im_end|>
        # <|im_start|>user
        # ![Brand Logo]({base64_logo})
        # Brand Description: {}
        # provide: Color palette insights and visual characteristics, marketing tone, brand personality traits
        # <|im_end|>
        # <|im_start|>assistant>
        # """
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 300,
                "temperature": 0.7
            }
        }
        response = requests.post(self.LLAVA_URL, headers=headers, json=payload)
        
        if response.status_code == 200:
            response_data = response.json()
            return response.json().get("generated_text", "").strip()
        else:
            raise Exception(f"Brand asset analysis failed: {response.text}")

In [5]:
class VisualGenerator:
    """
    Handles visual content generation using Flux API
    """
    def __init__(self, flux_token):
        #Replace you flux server
        self.FLUX_URL = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
        self.FLUX_TOKEN = flux_token

    def generate_socialMedia_visual(self, brand_insights, visual_theme,socail_Media):
        """
        Generate a professional visual 
        
        Args:
            brand_insights (str): Brand insights from analysis
            visual_theme (str): Theme for the visual
        
        Returns:
            bytes: Generated image content
        """
        headers = {
            "Authorization": f"Bearer {self.FLUX_TOKEN}",
            "Content-Type": "application/json"
        }
        # Incorporate brand insights into visual generation
        detailed_prompt = f""" Create a  modern, high quality {socail_Media} marketing image,{brand_insights} Visual Theme:{visual_theme}."""
        detailed_prompt=detailed_prompt[:200]

        print(detailed_prompt)
        payload = {
            "prompt": detailed_prompt,
            "img_size": 1024,
            "guidance_scale": 8.5,
            "num_inference_steps": 50
        }
        print("------- calling visual api")

        response = requests.post(self.FLUX_URL, headers=headers, json=payload)
        if response.status_code == 200:
            return response.content
        else:
            raise Exception(f"Visual generation failed: {response.text}")

In [6]:
class MarketingAI:
    """
    Orchestrator class that combines all components to create a marketing post
    """
    def __init__(self, phi3_token, flux_token, llava_token):
        self.brand_analyzer = BrandAnalyzer(llava_token)
        self.copy_generator = CopyGenerator(phi3_token)
        self.visual_generator = VisualGenerator(flux_token)

    def  create_post(self, brand_name, brand_description, brand_logo_path, post_objective, visual_theme, social_media):
        """
        Create a complete marketing post with multimodal insights
        
        Args:
            brand_name (str): Name of the brand
            brand_description (str): Description of the brand
            brand_logo_path (str): Path to brand logo image
            post_objective (str): Marketing post objective
            visual_theme (str): Theme for the visual
        
        Returns:
            dict: Complete marketing post with insights, copy, and image
        """
        # Analyze brand assets using LLava-Next
        brand_insights = self.brand_analyzer.analyze_brand_assets(brand_logo_path, brand_description)
        # Generate marketing copy
        marketing_copy = self.copy_generator.generate_marketing_copy(brand_insights, post_objective,social_media)
        # Generate visual
        visual_content = self.visual_generator.generate_socialMedia_visual(brand_insights, visual_theme,social_media)
        return {
            "brand_name": brand_name,
            "brand_insights": brand_insights,
            "marketing_copy": marketing_copy,
            "visual_content": visual_content
        }

In [7]:
class CopyGenerator:
    """
    Handles copy generation using Phi3 API
    """
    def __init__(self, phi3_token):
        #Replace your PHI3 Server URL
        self.PHI3_URL = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
        self.PHI3_TOKEN = phi3_token

    def generate_marketing_copy(self, brand_insights, post_objective,social_media):
        """
        Generate marketing copy using brand insights
        
        Args:
            brand_insights (str): Insights from brand asset analysis
            post_objective (str): Marketing post objective
        
        Returns:
            str: Generated marketing copy
        """
        headers = {
            "Authorization": f"Bearer {self.PHI3_TOKEN}",
            "Content-Type": "application/json"
        }
        
        prompt = f"""
        <|system|>
        You are a professional B2B marketing content writer for {social_media}.
        Include more emojis for {social_media} Post  for enagaement and Hashtags at the bottom
        Create a post that aligns with these brand insights:
        {brand_insights}      
        Post Objective: {post_objective}   
        Craft a post that uses a professional tone matching brand personality key business values and encourages professional engagement
        <|end|> 
        <|user|>
        Create a {social_media} marketing post that captures the brand essence.
        <|end|> 
        <|assistant|>
        """
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 300,
                "temperature": 0.7
            }
        }
        
        response = requests.post(self.PHI3_URL, headers=headers, json=payload)
        if response.status_code == 200:
            return response.json().get("generated_text", "").strip()
        else:
            raise Exception(f"Marketing copy generation failed: {response.text}")

In [8]:
def main():
    # Replace with actual tokens
    PHI3_TOKEN = ""
    FLUX_TOKEN = ""
    LLAVA_TOKEN = ""
    
    # Initialize the marketing AI
    marketing_ai = MarketingAI(PHI3_TOKEN, FLUX_TOKEN, LLAVA_TOKEN)
    
    # Example usage
    try:
        post = marketing_ai.create_post(
            brand_name="Dallas Fried Chicken",
            brand_description="Best Fried Chicken in Texas",
            brand_logo_path="./friedchicken.jpg", 
            social_media="Instagram",
            post_objective="Increase brand awareness in {social_media}",
            visual_theme="Cinematic, friendly"
        )
        

        print("---------------------------------------------------------------------------------")
        print("Marketing Post Generated Successfully!")
        print("\nBrand Insights:\n", post['brand_insights'])
        print("\nMarketing Copy:\n", post['marketing_copy'])

        final_image = ImageProcessor.overlay_text_on_image(
            post['visual_content'], 
            post['brand_insights'] + "..."
        )
        with open("marketing_post.png", "wb") as f:
            f.write(final_image)
        
    except Exception as e:
        print(f"Error generating marketing post: {e}")

In [9]:
if __name__ == "__main__":
    main()

 Create a  modern, high quality Instagram marketing image,This image shows a delicious-looking plate of fried chicken, accompanied by a beverage. The chicken appears to be golden brown and crispy, ind
------- calling visual api
---------------------------------------------------------------------------------
Marketing Post Generated Successfully!

Brand Insights:
 This image shows a delicious-looking plate of fried chicken, accompanied by a beverage. The chicken appears to be golden brown and crispy, indicating it may have been freshly cooked. It is placed on a white plate, which contrasts with the food and makes it look appetizing. The beverage is served in a clear glass, allowing the viewer to see the contents.

The fried chicken and beverage are placed on a dining table, which suggests that this could be a mealtime scene. The arrangement of the food and drink indicates that they are ready to be enjoyed.

Fried chicken is a popular dish in the southern United States, including Texas.